# __Self Correlation__ (R)

At its most fundamental, self-correlation of a time series is the idea that a value in a
time series at one given point in time may have a correlation to the value at another
point in time. 

As an example of self correlation, if you take a yearly time series of daily temperature
data, you may find that comparing May 15th of every year to August 15th of every
year will give you some correlation, such that hotter May 15ths tend to correlate with
hotter August 15ths (or tend to correlate with cooler August 15ths). You may feel you
have learned a potentially interesting fact about the temperature system, indicating
that there is a certain amount of long-term predictability. On the other hand, you
may find the correlation closer to zero, in which case you will also have found some‐
thing interesting, namely that knowing the temperature on May 15th does not alone
give you any information about the likely range of temperatures on August 15th.
That is self-correlation in an anecdotal nutshell.